In [11]:
import agate
import json

%store -r cpi_and_cl

In [8]:
country_json = json.loads(open('earth.json', 'rb').read())

country_dict = {}

for dct in country_json:
    country_dict[dct['name']] = dct['parent']

# given a contry row, it returns the continent.
def get_country(country_row):
    return country_dict.get(country_row['Country / Territory'].lower())

cpi_and_cl = cpi_and_cl.compute([('continent',
                                   agate.Formula(agate.Text(), get_country))
                                ])

for r in cpi_and_cl.rows:
    print(r['Country / Territory'], r['continent'])

Uruguay south america
Chile south america
Saint Lucia None
Botswana africa
Bhutan asia
Portugal europe
Costa Rica north america
Rwanda africa
Turkey europe
Georgia europe
Lesotho africa
Bahrain asia
Ghana africa
Jordan asia
Montenegro europe
Romania europe
Bosnia and Herzegovina None
Brazil south america
Sao Tome and Principe None
Serbia europe
Senegal africa
Tunisia africa
Swaziland africa
Burkina Faso africa
El Salvador north america
Jamaica north america
Liberia africa
Mongolia asia
Peru south america
Trinidad and Tobago None
Zambia africa
Malawi africa
Morocco africa
Algeria africa
Armenia europe
Benin africa
Colombia south america
Djibouti africa
India asia
Philippines None
Suriname south america
Ecuador south america
Panama north america
Thailand asia
Argentina south america
Gabon africa
Mexico north america
Niger africa
Ethiopia africa
Egypt africa
Indonesia asia
Albania europe
Nepal asia
Mauritania africa
Mozambique africa
Sierra Leone africa
Timor-Leste None
Belarus europe
Dom

C:\conda_tmp\ipykernel_15640\3985804638.py:1: ResourceWarning: unclosed file <_io.BufferedReader name='earth.json'>


In [9]:
# print out the countries that have no continent 'None'
no_continent = cpi_and_cl.where(lambda x: x['continent'] is None)

for r in no_continent.rows:
    print(r['Country / Territory'])

Saint Lucia
Bosnia and Herzegovina
Sao Tome and Principe
Trinidad and Tobago
Philippines
Timor-Leste
Democratic Republic of the Congo
Equatorial Guinea
Guinea-Bissau


In [16]:
%store -r cpi_and_cl

# earth-cleaned.json, which is the earth.json file with the necessary changes made.
country_json = json.loads(open('earth-cleaned.json', 'rb').read())

country_dict = {}

for dct in country_json:
    country_dict[dct['name']] = dct['parent']

# given a contry row, it returns the continent.
def get_country(country_row):
    return country_dict.get(country_row['Country / Territory'].lower())

cpi_and_cl = cpi_and_cl.compute([('continent',
                                   agate.Formula(agate.Text(), get_country))
                                ])

# print out the countries that have no continent 'None'
no_continent = cpi_and_cl.where(lambda x: x['continent'] is None)

for r in no_continent.rows:
    print(r['Country / Territory'])

C:\conda_tmp\ipykernel_15640\3835179319.py:4: ResourceWarning: unclosed file <_io.BufferedReader name='earth-cleaned.json'>


In [ ]:
# group our now-complete continent data by continent